# RRTMGP_RFMIP_LW

*Last edited: 2024-10-18*

This code is part of RRTM for GCM Applications - Parallel (RRTMGP). Please see `rrtmgp_rfmip_lw.F90` for more info.

- Example program to demonstrate the calculation of longwave radiative fluxes in clear, aerosol-free skies.
- The example files come from the Radiative Forcing MIP (https://www.earthsystemcog.org/projects/rfmip/).
- The large problem (1800 profiles) is divided into blocks.
- Program is invoked as rrtmgp_rfmip_lw [block_size input_file coefficient_file upflux_file downflux_file] .
- All arguments are optional but need to be specified in order.

In [2]:
WORKDIR = "/home/x/git/pd1b24/rrtmgp/ukk23test01/examples/rfmip-clear-sky/"
%cd {WORKDIR}

/home/x/git/pd1b24/rrtmgp/ukk23test01/examples/rfmip-clear-sky


## Makefile

For `rrtmgp_rfmip_lw.F90` and `rrtmgp_rfmip_sw.F90`

In [8]:
%%writefile Makefile
# Set k-distribution files
# KDIST_LW=rrtmgp-data-lw-g256-2018-12-04.nc
# KDIST_SW=rrtmgp-data-sw-g224-2018-12-04.nc
KDIST_LW=rrtmgp-data-lw-g128-210809.nc
KDIST_SW=rrtmgp-data-sw-g112-210809.nc
BLOCKSIZE = 36

# Here set variables RRTMGP_BUILD, NCHOME, NFHOME, TIME_DIR (for GPTL)
# and link against BLAS library, or have those variables set in the
# environment

RRTMGP_ROOT=../../
$(info VAR="$(RRTMGP_ROOT)")
RRTMGP_BUILD=$(RRTMGP_ROOT)/build
-include $(RRTMGP_BUILD)/Makefile.conf

# RRTMGP library, module files

LDFLAGS   += -L$(RRTMGP_BUILD)
LIBS      += -lrrtmgp -lrte -lneural
FCINCLUDE += -I$(RRTMGP_BUILD)

# netcdf library, module files
# C and Fortran interfaces respectively

FCINCLUDE += -I$(NFHOME)/include
LDFLAGS   += -L$(NFHOME)/lib -L$(NCHOME)/lib
LIBS      += -lnetcdff -lnetcdf

# -------- BLAS lbirary: required for RRTMGP-NN -------- 
# Choose BLAS library: e.g. "blis", "openblas", "mkl".
# Here we use BLIS (https://github.com/flame/blis) which is one of the
# fastest BLAS libraries available (on Intel platforms MKL is probably
# faster). The flags BLAS_DIR and BLASLIB can be specified in
# Makefile.conf or the environment Depending on your BLAS library you
# might have to make changes below.

ifeq ($(BLASLIB),blis)
	FCINCLUDE   += -I$(BLAS_DIR)/include/blis
	LIBS      	+= $(BLAS_DIR)/lib/libblis.a -lm -lpthread 
else ifeq ($(BLASLIB),blis-amd)
	FCINCLUDE   += -I$(BLAS_DIR)/include/blis
	LIBS      	+= $(BLAS_DIR)/lib/libblis-mt.a -lm -lpthread 
else ifeq ($(BLASLIB),openblas)
	LIBS 		+= -lopenblas 
else ifeq ($(BLASLIB),mkl)
	FCINCLUDE   += -I${MKLROOT}/include
	#LDFLAGS     += -L${MKLROOT}/lib/intel64 -lmkl_rt
	#LDFLAGS     += -lpthread -lm -ldl
endif

# -------- Use OpenMP? May be needed for BLAS -------
# ifeq ($(BLASLIB),blis-amd)
#    USE_OPENMP=1
# endif
#USE_OPENMP=1

ifeq ($(USE_OPENMP),1)
	# OMPFLAG = -fopenmp
	FCFLAGS += -DUSE_OPENMP
endif
OMPFLAG = -fopenmp

ifeq ($(USE_OPENACC),1)
  FCFLAGS += -DUSE_OPENACC
endif

# -------- GPTL Timing -------
# Use General Purpose Timing Library to profile code?

ifeq ($(GPTL_TIMING),1)

	# Timing library

	FCINCLUDE += -I$(TIME_DIR)/include

    # Compiler specific

	FCFLAGS += -DUSE_TIMING
	LDFLAGS   += -L$(TIME_DIR)/lib -Wl,-rpath=$(TIME_DIR)/lib
	LIBS      += -lgptl

# Use GPTL together with PAPI hardware instrumentation to measure 
# computational intensity? Requires that your GPTL was compiled to PAPI 
# support

else ifeq ($(GPTL_TIMING),2)

	# Timing library

	FCINCLUDE += -I$(TIME_DIR)/include

	# Compiler specific

	FCFLAGS += -DUSE_TIMING -DUSE_PAPI
	LDFLAGS   += -L$(TIME_DIR)/lib -Wl,-rpath=$(TIME_DIR)/lib

	# LIBS      += -lgptl  -rdynamic  -lpapi

	LIBS      += -lgptl -lpapi

    ifeq ($(FC),gfortran)
    	LIBS      += -rdynamic
    endif

endif

VPATH = ../

# Compilation rules
%.o: %.F90
	$(FC) $(FCFLAGS) $(FCINCLUDE) -c $< $(OMPFLAG)

%: %.o
	$(FC) $(FCFLAGS) -o $@ $^ $(LDFLAGS) $(LIBS) $(OMPFLAG)

#
# Ancillary codes
#
ADDITIONS = mo_simple_netcdf.o mo_rfmip_io.o mo_load_coefficients.o
all: rrtmgp_rfmip_lw rrtmgp_rfmip_sw
rrtmgp_rfmip_lw:   rrtmgp_rfmip_lw.o   $(ADDITIONS) $(RRTMGP_BUILD)/librte.a $(RRTMGP_BUILD)/librrtmgp.a $(RRTMGP_BUILD)/libneural.a
rrtmgp_rfmip_lw.o: rrtmgp_rfmip_lw.F90 $(ADDITIONS)
rrtmgp_rfmip_sw:   rrtmgp_rfmip_sw.o   $(ADDITIONS) $(RRTMGP_BUILD)/librte.a $(RRTMGP_BUILD)/librrtmgp.a $(RRTMGP_BUILD)/libneural.a
rrtmgp_rfmip_sw.o: rrtmgp_rfmip_sw.F90 $(ADDITIONS)
mo_rfmip_io.o:          mo_rfmip_io.F90          mo_simple_netcdf.o
mo_load_coefficients.o: mo_load_coefficients.F90 mo_simple_netcdf.o

tests: multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc # Also the template files
	# Files need to have been generated/downloaded before
	$(RUN_CMD) ./rrtmgp_rfmip_lw $(BLOCKSIZE) multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc ${RRTMGP_ROOT}/rrtmgp/data/$(KDIST_LW)
	$(RUN_CMD) ./rrtmgp_rfmip_sw $(BLOCKSIZE) multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc ${RRTMGP_ROOT}/rrtmgp/data/$(KDIST_SW)

check:
	cd ${RRTMGP_ROOT}/examples/rfmip-clear-sky && python ./compare-to-reference.py --fail=7.e-4

multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc:
	bash ./stage_files.sh

clean:
	-rm rrtmgp_rfmip_sw rrtmgp_rfmip_lw *.o *.mod *.optrpt

Writing Makefile


## rrtmgp_rfmip_lw.F90

- Notes:
    - OpenMP and OpenACC will not be used in this version, and have been removed from the code to simplify and facilitate understanding.

Start of the code

- Error checking: Procedures in rte+rrtmgp return strings which are empty if no errors occured.
- Check the incoming string, print it out and stop execution if non-empty.

In [8]:
%%writefile rrtmgp_rfmip_lw.F90

subroutine stop_on_err(error_msg)
  use iso_fortran_env, only : error_unit
  use iso_c_binding
  character(len=*), intent(in) :: error_msg

  if(error_msg /= "") then
    write (error_unit,*) trim(error_msg)
    write (error_unit,*) "rrtmgp_rfmip_lw stopping"
    stop
  end if
end subroutine stop_on_err

Overwriting helloworld.f90


### Main program

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

program rrtmgp_rfmip_lw

### Modules for working with rte and rrtmgp

 Working precision for real variables:

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

use mo_rte_kind, only: wp, sp, wl, i4

- Optical properties of the atmosphere as array of values
  - In the longwave we include only absorption optical depth (_1scl)
  - Shortwave calculations would use optical depth, single-scattering albedo, asymmetry parameter (_2str)

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

use mo_optical_props, only: ty_optical_props_1scl

- Gas optics: maps physical state of the atmosphere to optical properties

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

use mo_gas_optics_rrtmgp, only: ty_gas_optics_rrtmgp

- Gas optics uses a derived type to represent gas concentrations compactly...

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

 use mo_gas_concentrations, only: ty_gas_concs

- ... and another type to encapsulate the longwave source functions.

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

use mo_source_functions, only: ty_source_func_lw

- RTE longwave driver.

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

use mo_rte_lw, only: rte_lw

- RTE driver uses a derived type to reduce spectral fluxes to whatever the user wants
  - Here we're just reporting broadband fluxes

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

use mo_fluxes, only: ty_fluxes_broadband, ty_fluxes_flexible

  - modules for reading and writing files
  - RRTMGP's gas optics class needs to be initialized with data read from a netCDF files

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

use mo_load_coefficients, only: load_and_init
use mo_rfmip_io,          only: read_size, read_and_block_pt, read_and_block_gases_ty,  &
                                unblock_and_write, unblock, read_and_block_lw_bc, determine_gas_names
use mo_simple_netcdf,     only: read_field, get_dim_size
use netcdf
use mod_network_rrtmgp

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

In [ ]:
%%writefile -a rrtmgp_rfmip_lw.F90

In [11]:
! gfortran helloworld.f90

In [12]:
! ./a.out

 Hello, world!
